In [1]:
#!pip install pandas numpy emoji langdetect transformers torch


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import csv
import os
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.cluster import KMeans
import numpy as np
import torch
import torch.nn.functional as F
import urllib.request
from scipy.special import softmax

c:\Users\HRMic\OneDrive - ADvTECH Ltd\Documents\VS code\Python_VS\TWITTER\Research\gpuEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('2024_Classified_Balanced.csv')
df.head()

,label,tweet
0,Left,@ USER Fuck Joe Biden fuck Ukraine fuck the De...
1,Right,Fly them back to their home country just like ...
2,Right,Wow ! I ca n't believe the @ USER . They 're s...
3,Right,@ USER @ USER @ USER The election be rig again...
4,Right,@ USER @ USER @ USER @ USER @ USER Why be acto...


## **Roberta Sentiment MODEL**
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [4]:
# load model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, use_safetensors=True).to('cuda')

# load label mapping
mapping_link = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode("utf-8").split("\n")
    csvreader = csv.reader(html, delimiter="\t")
labels = [row[1] for row in csvreader if len(row) > 1]
print("Labels:", labels)
# (Barbieri et al., 2020; cardiffnlp, 2020)

c:\Users\HRMic\OneDrive - ADvTECH Ltd\Documents\VS code\Python_VS\TWITTER\Research\gpuEnv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HRMic\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for thi

Labels: ['negative', 'neutral', 'positive']


In [5]:
# preprocess text (username and link replacement)
# to follow consistency with the model's training data
def preprocess(text): # (Barbieri et al., 2020; cardiffnlp, 2020)
    new_text = []
    for t in str(text).lower().split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [6]:
# function to get sentiment
def getSentiment(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to('cuda')
    with torch.no_grad():
        output = model(**encoded_input)
        scores = output.logits[0].detach().cpu().numpy()
        scores = softmax(scores)
    pred_id = np.argmax(scores)
    # return label and scores
    return labels[pred_id], scores
# (Barbieri et al., 2020; cardiffnlp, 2020)

In [8]:
# apply the function to the DataFrame
sentOutput = df["tweet"].apply(getSentiment)
# split tuple results into separate columns
df["sentiment_label"] = sentOutput.apply(lambda x: x[0])
df["sentiment_probs"] = sentOutput.apply(lambda x: x[1])
df.head()
# (Barbieri et al., 2020; cardiffnlp, 2020)

,label,tweet,sentiment_label,sentiment_probs
0,Left,@ USER Fuck Joe Biden fuck Ukraine fuck the De...,negative,"[0.92917615, 0.06484142, 0.0059823976]"
1,Right,Fly them back to their home country just like ...,negative,"[0.5898782, 0.3819204, 0.028201416]"
2,Right,Wow ! I ca n't believe the @ USER . They 're s...,negative,"[0.9500361, 0.044342536, 0.005621288]"
3,Right,@ USER @ USER @ USER The election be rig again...,negative,"[0.888374, 0.10098925, 0.010636724]"
4,Right,@ USER @ USER @ USER @ USER @ USER Why be acto...,negative,"[0.70596755, 0.28732908, 0.0067033595]"


In [9]:
labels = ['negative', 'neutral', 'positive']
sentiment_counts = df.groupby(['label', 'sentiment_label']).size().unstack(fill_value=0)
# convert counts to proportions for each political leaning
sentiment_avg = sentiment_counts.div(sentiment_counts.sum(axis=1), axis=0)

In [ ]:
# Save results
df.to_csv("2024_sentiment_robertaBase.csv", index=False)

**Results**

In [11]:
# average sentiment probabilities by political leaning
print("\nAverage Sentiment Distribution by Political Leaning:\n")
sentiment_avg



Average Sentiment Distribution by Political Leaning:



sentiment_label,negative,neutral,positive
label,,,
Left,0.484969,0.348419,0.166613
Right,0.548921,0.370285,0.080793
